## Intent classifier trainer

* Dataset: student_queries.csv file with synthetic messages, each labeled with one of the intent categories, such as: Course Information, Enrollment / Course Registration, Withdrawal or Drop Course, Scholarship/Financial Aid, etc.
* Task: Single-label classification to predict the intent behind the student's query.
* Model: Use BertForSequenceClassification, fine-tuned on the intent dataset.
* Goal: To determine the purpose of a query.
* The trained model is saved to models/intentClassifier.

In [27]:
intent_labels = [
        "Course Information",
        "Enrollment / Course Registration",
        "Withdrawal or Drop Course",
        "Access Issues (portal/login)",
        "Technical Support",
        "Tuition/Fees Inquiry",
        "Scholarship/Financial Aid",
        "Mental Health Concerns",
        "Stress or Burnout",
        "Bullying or Harassment",
        "Administrative Support",
        "Campus Facilities",
        "Housing/Accommodation",
        "Extracurricular Activities",
        "General Complaint"
    ]

In [ ]:
import pandas as pd

label_to_id = {label: idx for idx, label in enumerate(intent_labels)}
id_to_label = {idx: label for idx, label in enumerate(intent_labels)}

df = pd.read_csv("../data/student_queries.csv")

# Encode labels
df["label"] = df["intent"].map(label_to_id)
df = df.dropna(subset=["label"])

# convert label to int (from float due to NaN)
df["label"] = df["label"].astype(int)

In [29]:
df.head()

,datetime,student,question,intent,is_distressed,label
0,2025-07-04 14:07:23,Shannon Austin,What topics will be covered in the AI course?,Course Information,False,0
1,2025-07-04 16:13:28,Stephanie Calhoun,I'm feeling really overwhelmed lately.,Mental Health Concerns,True,7
2,2025-07-04 16:19:28,Kevin Garcia,Who do I contact for transcript requests?,Administrative Support,False,10
3,2025-07-04 16:55:20,Lisa Duran,I have a complaint about the cafeteria service.,General Complaint,True,14
4,2025-07-04 17:08:35,Jeff Rangel,Who do I contact for transcript requests?,Administrative Support,False,10


In [30]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[["question", "label"]])
test_dataset = Dataset.from_pandas(test_df[["question", "label"]])

In [31]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["question"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Model setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(intent_labels)
)

# Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Map: 100%|██████████| 615/615 [00:00<00:00, 10244.55 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/hsi/1557code/FML/PROG824_CSCN8010-Final_Project_Dry_Run/.venvpj/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
# Train the model
trainer.train()

# Evaluate on test set
trainer.evaluate()

  0%|          | 0/1540 [00:00<?, ?it/s]/Users/hsi/1557code/FML/PROG824_CSCN8010-Final_Project_Dry_Run/.venvpj/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  6%|▋         | 100/1540 [00:37<08:15,  2.90it/s]

{'loss': 2.0215, 'grad_norm': 6.998689651489258, 'learning_rate': 1.8701298701298704e-05, 'epoch': 0.65}


 10%|█         | 154/1540 [00:58<08:12,  2.81it/s]

{'eval_loss': 0.47589096426963806, 'eval_accuracy': 0.9853658536585366, 'eval_f1': 0.9854004377297552, 'eval_precision': 0.9858250276854928, 'eval_recall': 0.9853658536585366, 'eval_runtime': 1.7907, 'eval_samples_per_second': 343.44, 'eval_steps_per_second': 5.584, 'epoch': 1.0}


/Users/hsi/1557code/FML/PROG824_CSCN8010-Final_Project_Dry_Run/.venvpj/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 13%|█▎        | 200/1540 [01:16<07:36,  2.93it/s]

...

{'eval_loss': 0.04177943989634514, 'eval_accuracy': 0.9869918699186991, 'eval_f1': 0.9869696007758829, 'eval_precision': 0.9871835075493612, 'eval_recall': 0.9869918699186991, 'eval_runtime': 1.8538, 'eval_samples_per_second': 331.753, 'eval_steps_per_second': 5.394, 'epoch': 5.0}


 50%|█████     | 770/1540 [04:56<04:56,  2.60it/s]
/Users/hsi/1557code/FML/PROG824_CSCN8010-Final_Project_Dry_Run/.venvpj/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 296.3664, 'train_samples_per_second': 83.005, 'train_steps_per_second': 5.196, 'train_loss': 0.37213142839345065, 'epoch': 5.0}


100%|██████████| 10/10 [00:01<00:00,  6.38it/s]


{'eval_loss': 0.04414517059922218,
 'eval_accuracy': 0.9902439024390244,
 'eval_f1': 0.9902409992704504,
 'eval_precision': 0.9905500184569951,
 'eval_recall': 0.9902439024390244,
 'eval_runtime': 1.9358,
 'eval_samples_per_second': 317.696,
 'eval_steps_per_second': 5.166,
 'epoch': 5.0}

In [38]:
trainer.save_model("../models/intentClassifier")
tokenizer.save_pretrained("../models/intentClassifier")

('../models/intentClassifier/tokenizer_config.json',
 '../models/intentClassifier/special_tokens_map.json',
 '../models/intentClassifier/vocab.txt',
 '../models/intentClassifier/added_tokens.json')

In [34]:
# Inference example
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class_id = outputs.logits.argmax().item()
    return intent_labels[predicted_class_id]

If you want to equipment using MPS, please execute this code.

In [ ]:
# import torch

# # Check if MPS is available and set the device
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# print(f"Equipment used: {device}")

# # Assuming your model and tokenizer are defined elsewhere
# # For example:
# # from transformers import AutoTokenizer, AutoModelForSequenceClassification
# # tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# # model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# # Move the model to the specified device
# model.to(device)

# def predict_intent(text):
#     # Tokenize the input text
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
#     # Ensure all input tensors are moved to the same device as the model
#     # This step is critical to avoid the RuntimeError
#     inputs = {key: val.to(device) for key, val in inputs.items()}
    
#     # Perform inference
#     with torch.no_grad():
#         outputs = model(**inputs)
#         predicted_class_id = outputs.logits.argmax().item()
#     return intent_labels[predicted_class_id]

Equipment used: mps


In [39]:
# Example usage
print(predict_intent("How do I apply for scholarships?"))
print(predict_intent("I have a complaint about the cafeteria service?"))
print(predict_intent("Are there any upcoming student events"))

Scholarship/Financial Aid
General Complaint
Extracurricular Activities


In [40]:
print(predict_intent("When is the tuition payment deadline?"))
print(predict_intent("Hi, I'm trying to figure out how to pay my tuition fees."))

Tuition/Fees Inquiry
Scholarship/Financial Aid
